In [195]:
import numpy as np
import pandas as pd
import re
import string                             

!pip install tashaphyne
import tashaphyne
from tashaphyne.stemming import ArabicLightStemmer

!pip install pyarabic
import pyarabic

!pip install emoji
import emoji

import nltk                             
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords 
nltk.download('stopwords')                                
nltk.download('punkt')

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


[nltk_data] Downloading package stopwords to /home/ahmed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ahmed/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Preprocessing

In [196]:
# read csv file
df = pd.read_csv('Dataset/train.csv', encoding='UTF-8')
tweets = df['text']
categories = df['category']
stances = df['stance']

In [197]:
ArListem = ArabicLightStemmer()

def stem_tweet(tweet_tokens):
    for i in range(len(tweet_tokens)):
        ArListem.light_stem(tweet_tokens[i])
        tweet_tokens[i] = ArListem.get_root()
    return tweet_tokens

def remove_url(tweet):
    tweet = re.sub(r"http\S+", "", tweet)
    return tweet

def remove_retweet_tag(tweet):
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    return tweet

def remove_tweet_mentions(tweet):
    tweet = re.sub(r'@\w+', '', tweet)
    return tweet

def remove_text_control_tags(tweet):
    tweet = re.sub(r'\n|\t|\r|<LF>|<lf>', '', tweet)
    return tweet

def tokenize_tweet(tweet):
    tweet_tokens = re.split(', |_|-|!', tweet)
    return tweet_tokens

def remove_stopwords_punctuation(tweet_tokens, stop_words = ()):
    tweet_reduced = []
    for word in tweet_tokens:
        if (word not in stop_words and word not in string.punctuation):
            tweet_reduced.append(word)
            tweet_reduced[-1] = re.sub(r'[~`!@#$%^&*()-_+={}[\]|/\:;"`<>,.?؟،]+', '', tweet_reduced[-1])
    return tweet_reduced

def remove_specialcharacters(tweet):
    tweet = re.sub(r'#', '', tweet)
    return tweet

def handle_emojis(tweet):
    tweet = emoji.demojize(tweet, language='en') # convert emojis to text ENGLISH!!
    return tweet

def normalize_tweet(tweet_tokens):
    for i in range(len(tweet_tokens)):
        tweet_tokens[i] = pyarabic.araby.strip_tashkeel(tweet_tokens[i])
        tweet_tokens[i] = pyarabic.araby.strip_tatweel(tweet_tokens[i])
        tweet_tokens[i] = pyarabic.araby.strip_lastharaka(tweet_tokens[i])
        tweet_tokens[i] = pyarabic.araby.normalize_alef(tweet_tokens[i])
        tweet_tokens[i] = re.sub(r'(.)\1{3,}', r"\1\1\1", tweet_tokens[i]) # Remove longation
        tweet_tokens[i] = pyarabic.araby.normalize_hamza(tweet_tokens[i])
    return tweet_tokens


def preprocess_tweet(tweet):
    tweet = remove_url(tweet)
    tweet = remove_retweet_tag(tweet)
    tweet = remove_tweet_mentions(tweet)
    tweet = remove_specialcharacters(tweet)
    tweet = remove_text_control_tags(tweet)
    tweet = handle_emojis(tweet)
    tweet_tokens = tokenize_tweet(tweet)
    tweet_tokens = normalize_tweet(tweet_tokens)
    tweet_tokens_reduced = remove_stopwords_punctuation(tweet_tokens, stopwords.words('arabic'))
    # tweet_tokens_stemmed = stem_tweet(tweet_tokens_reduced)
    tweet = ' '.join(tweet_tokens_reduced)
    return tweet